# 2.model_structure

In [1]:
import sys
sys.path = ["../../.."] + sys.path # 切换到项目目录下

import scanpy as sc
import scvelo as scv
import velovgi

from ray import tune, air
from ray.air import session

Global seed set to 0
/usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


1. 目标函数

In [2]:
from pytorch_lightning import loggers
from torch_geometric import seed_everything

# TODO: 跳整多种参数，简化调参Trail的名字
def train_velovgi(config):
    # 提取参数
    # 随机数种子，确保结果的可复现性
    random_seed = config.get("random_seed", 0)
    # 预处理的参数
    n_bnn_neighbors = config.get("n_bnn_neighbors", 15)
    n_knn_neighbors = config.get("n_knn_neighbors", 15)
    is_ot = config.get("is_ot", True)
    # 模型结构参数
    n_hidden = config.get("n_hidden", 256)
    n_latent = config.get("n_latent", 10)
    n_layers = config.get("n_layers", 1)
    # 训练参数
    num_neighbors = [config.get("num_neighbors", 8)]*n_layers
    max_epochs = config.get("max_epochs", 10) # TODO:这里是最关键的一个参数，小epochs测试之后再提交到服务器上用大epoch
    batch_size = config.get("batch_size", 64)
    max_kl_weight = config.get("max_kl_weight", 0.8)

    name = ""
    for k,v in config.items():
        name += "%s_%s,"%(k, v)
    name = name[:-1]

    # seed_everything(random_seed)
    # # TODO:数据读入，对于不同的数据集这里需要替换
    # adata_filename = "/mnt/h/F_bak/Python进阶/scRNA/Other/velovgi_workstation/notebook/local_pc/erythroid_lineage/data/erythroid_lineage.h5ad" # 数据路径使用绝对路径
    # adata = scv.read(adata_filename)
    # batch_key = "stage" # 批次key
    cluster_key = "celltype" # 细胞类型key
    cluster_edges = [
    ("Blood progenitors 1", "Blood progenitors 2"), 
    ("Blood progenitors 2", "Erythroid1"), 
    ("Erythroid1", "Erythroid2"), 
    ("Erythroid2", "Erythroid3")
    ] # 指定对应数据集已知的细胞类型间的分化信息

    # # TODO:预处理，这里batch_pair_list以后可能需要手动指定
    # batch_list = list(adata.obs[batch_key].cat.categories)
    # batch_pair_list = list(zip(batch_list[:-1], batch_list[1:]))
    # subsample_adata = velovgi.pp.preprocess(adata,
    #                                         n_bnn_neighbors=n_bnn_neighbors,
    #                                         n_knn_neighbors=n_knn_neighbors,
    #                                         batch_key=batch_key,
    #                                         batch_pair_list=batch_pair_list,
    #                                         is_ot=is_ot)
    # TODO: 如果需要调整预处理之后的参数，就不需要重复做预处理了，读取预处理之后的结果即可
    adata = velovgi.tl.read_adata("/mnt/h/F_bak/Python进阶/scRNA/Other/velovgi_workstation/notebook/local_pc/erythroid_lineage/data/adata")
    subsample_adata = scv.read("/mnt/h/F_bak/Python进阶/scRNA/Other/velovgi_workstation/notebook/local_pc/erythroid_lineage/data/subsample_adata.h5ad") # 使用这个AnnData做训练
    seed_everything(random_seed)

    # 模型训练
    logger = loggers.TensorBoardLogger(save_dir="./log", name=name)
    velovgi.tl.VELOVGI.setup_anndata(adata=subsample_adata, spliced_layer="Ms", unspliced_layer="Mu")
    velovgi_model = velovgi.tl.VELOVGI(subsample_adata,
                                       n_hidden=n_hidden,
                                       n_latent=n_latent,
                                       n_layers=n_layers)
    velovgi_model.train(num_neighbors=num_neighbors,
                        max_epochs=max_epochs,
                        batch_size=batch_size,
                        plan_kwargs={"max_kl_weight": max_kl_weight},
                        logger=logger)

    # 模型恢复
    velovgi.tl.add_velovi_outputs_to_adata(subsample_adata, velovgi_model) # 模型输出
    velovgi.pp.moment_recover(adata, subsample_adata) # 恢复

    # 速率计算
    scv.tl.velocity_graph(adata)
    scv.pl.velocity_embedding(adata, color=cluster_key, title=name, save="arrow.png")
    scv.pl.velocity_embedding_stream(adata, color=cluster_key, title=name, legend_loc="right", save="stream.png")

    # 伪时间计算
    scv.tl.velocity_pseudotime(adata)
    scv.pl.velocity_embedding_stream(adata, color="velocity_pseudotime", title=name, colorbar=False, save="pseudotime.png")


    # 保存结果
    subsample_adata.write("subsample_adata.h5ad")
    velovgi.tl.write_adata(adata, "adata")
    velovgi_model.save("model")

    # 计算指标评价
    adata_velo = velovgi.tl.pre_metric(adata)
    exp_metrics = velovgi.tl.summary_metric(adata_velo, cluster_edges, cluster_key)[-1] # 计算指标汇总后的结果

    session.report({"CBDir": exp_metrics["CBDir"], "ICVCoh": exp_metrics["ICVCoh"]})


2. 搜索空间，这里可以添加键值，实现更多层面的网格调参

In [3]:
search_space = {
    # "num_neighbors" : tune.grid_search([2, 4]),
    "max_epochs" : tune.grid_search([10, 20, 30]),
    # "batch_size" : tune.grid_search([64, 128]),
    # "max_kl_weight" : tune.grid_search([0.6, 0.8]),
}

3. 执行调参，等待传入实验名称和搜索空间

In [4]:
from ray.tune.schedulers import ASHAScheduler

name = "model_train" # TODO:指定此次调参的名字，这里是预处理过程的调参

tuner = tune.Tuner(
    train_velovgi,
    tune_config=tune.TuneConfig(
        metric="CBDir",
        mode="max",
        scheduler=ASHAScheduler()
    ),
    run_config=air.RunConfig(
        local_dir="./results", # Trail内部具体输出结果在这里保存
        name=name # 开启调参的Tensorboard日志
    ),
    param_space=search_space,
)

results = tuner.fit()

2023-06-10 11:10:37,793	INFO worker.py:1625 -- Started a local Ray instance.
2023-06-10 11:10:39,625	INFO tune.py:218 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2023-06-10 11:10:39,652	INFO tensorboardx.py:172 -- pip install "ray[tune]" to see TensorBoard files.
2023-06-10 11:10:39,652	WARNING callback.py:142 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(pid=5185) Global seed set to 0
(pid=5185) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
(pid=5185)   new_rank_zero_deprecation(
(pid=5185) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
(pid=5185)   return new_rank_zero_deprecation(*args, **kwargs)


(train_velovgi pid=5185) load /mnt/h/F_bak/Python进阶/scRNA/Other/velovgi_workstation/notebook/local_pc/erythroid_lineage/data/adata/adata.h5ad
(train_velovgi pid=5185) load /mnt/h/F_bak/Python进阶/scRNA/Other/velovgi_workstation/notebook/local_pc/erythroid_lineage/data/adata/sample_recover.pkl
(train_velovgi pid=5185) 初始训练，初始化runner参数
(train_velovgi pid=5185) choosing neighbor minibatch


(train_velovgi pid=5185) GPU available: False, used: False
(train_velovgi pid=5185) TPU available: False, using: 0 TPU cores
(train_velovgi pid=5185) IPU available: False, using: 0 IPUs
(train_velovgi pid=5185) HPU available: False, using: 0 HPUs
(train_velovgi pid=5185) Missing logger folder: ./log/max_epochs_10


Epoch 3/10:  20%|██        | 2/10 [00:03<00:12,  1.61s/it, loss=1.87e+06, v_num=0]


...

Trial name,CBDir,ICVCoh,date,done,experiment_tag,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
train_velovgi_60adc_00000,0.535893,0.971254,2023-06-10_11-11-51,True,0_max_epochs=10,DESKTOP-9GVJMSD,1,172.29.205.215,5185,66.0054,66.0054,66.0054,1686366711,1,60adc_00000
train_velovgi_60adc_00001,0.765104,0.972532,2023-06-10_11-12-16,True,1_max_epochs=20,DESKTOP-9GVJMSD,1,172.29.205.215,5250,84.8688,84.8688,84.8688,1686366736,1,60adc_00001
train_velovgi_60adc_00002,0.842325,0.970403,2023-06-10_11-12-37,True,2_max_epochs=30,DESKTOP-9GVJMSD,1,172.29.205.215,5316,97.5078,97.5078,97.5078,1686366757,1,60adc_00002


Epoch 30/30: 100%|██████████| 30/30 [01:04<00:00,  2.13s/it, loss=1.15e+06, v_num=0]


(train_velovgi pid=5316) `Trainer.fit` stopped: `max_epochs=30` reached.


(train_velovgi pid=5250) computing velocity graph (using 1/12 cores)
(train_velovgi pid=5250)   0%|          | 0/500 [00:00<?, ?cells/s]
(train_velovgi pid=5250)     finished (0:00:01) --> added 
(train_velovgi pid=5250)     'velocity_graph', sparse matrix with cosine correlations (adata.uns)
(train_velovgi pid=5250) computing velocity embedding
(train_velovgi pid=5250)     finished (0:00:00) --> added
(train_velovgi pid=5250)     'velocity_umap', embedded velocity vectors (adata.obsm)
(train_velovgi pid=5250) saving figure to file ./figures/scvelo_arrow.png
(train_velovgi pid=5250) Figure(640x480)
(train_velovgi pid=5250) saving figure to file ./figures/scvelo_stream.png
(train_velovgi pid=5250) Figure(640x480)
(train_velovgi pid=5250) saving figure to file ./figures/scvelo_pseudotime.png
(train_velovgi pid=5250) Figure(640x480)
(train_velovgi pid=5250) create adata
(train_velovgi pid=5250) save adata/sample_recover.pkl
(train_velovgi pid=5250) save adata/adata.h5ad
(train_velovgi pid

2023-06-10 11:12:37,778	INFO tune.py:945 -- Total run time: 118.15 seconds (118.06 seconds for the tuning loop).
